In [ ]:
import pandas as pd
import json
import time
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon

In [ ]:
caminho_data = 'data_final.csv'
caminho_json = 'C:/Users/abner/Desktop/python/Polygon/dados_IBGE/Coordendas_municipios.geojson'
caminho_dados_mun = 'AR_BR_RG_UF_MES_MIC_MUN_2017.xlsx'

In [ ]:
dados_mun = pd.read_excel(caminho_dados_mun, encoding='latin-1')

In [ ]:
with open(caminho_json, encoding="latin-1") as file:
    coords_json = json.load(file)

In [ ]:
# coords_json['features'][1067]['properties']['CD_GEOCMU']
# coords_json['features'][1067]['geometry']['coordinates']

In [ ]:
data_final = pd.DataFrame()
data_final = pd.DataFrame(columns=['COD_UF','CD_GEOCMU','UF','ESTADO','CIDADE','COORDENADAS'])
for i in range(0, len(coords_json['features'])):

    data_final.loc[i,'COD_UF'] = coords_json['features'][i]['properties']['CD_GEOCMU'][0:2]
    data_final.loc[i,'CD_GEOCMU'] = coords_json['features'][i]['properties']['CD_GEOCMU']
    
    data_final.loc[i,'UF'] = dados_mun.iloc[ dados_mun.loc[dados_mun['CD_GCMUN'] == int(data_final.loc[i,'CD_GEOCMU']) ].index[0]  ]['NM_UF_SIGLA'] 
    
    data_final.loc[i,'ESTADO'] = dados_mun.iloc[ dados_mun.loc[dados_mun['CD_GCMUN'] == int(data_final.loc[i,'CD_GEOCMU']) ].index[0]  ]['NM_UF'].upper()
    data_final.loc[i,'CIDADE'] = dados_mun.iloc[ dados_mun.loc[dados_mun['CD_GCMUN'] == int(data_final.loc[i,'CD_GEOCMU']) ].index[0]  ]['NM_MUN_2017'].upper()
#     data_final.loc[i,'COORDENADAS'] = str(coords_json['features'][i]['geometry']['coordinates']).replace('[[[','[[').replace(']]]',']]').replace('[[','[').replace(']]',']').replace(']','').replace('[','').replace(' ','') 
    data_final.loc[i,'COORDENADAS'] = coords_json['features'][i]['geometry']['coordinates'] 


In [ ]:
# for i,chunk in enumerate(pd.read_csv('data_final.csv', chunksize=1800,encoding='latin-1')):
#     chunk.to_csv('chunk{}.csv'.format(i))

In [ ]:
data_final.to_csv('data_final.csv', encoding='latin-1')

In [ ]:
coordenadas_cidades = pd.DataFrame(columns=['CD_GEOCMU','ESTADO','CIDADE','COORDENADAS'])
count = 0
for i in range (0, data_final.shape[0]):
# for i in range (0, 10):
    split_coords = str(data_final.COORDENADAS[i]).replace(" ","").split(']],')
                       
    for k in range (0, len(split_coords)):
        coordenadas_cidades.loc[count,'CD_GEOCMU'] = data_final.CD_GEOCMU[i]
        coordenadas_cidades.loc[count,'ESTADO'] = data_final.ESTADO[i] 
        coordenadas_cidades.loc[count,'CIDADE'] = data_final.CIDADE[i] 
        
        coordenadas_cidades.loc[count,'COORDENADAS'] = split_coords[k].replace('[[[','[[').replace(']]]',']]').replace('[[','[').replace(']]',']').replace(']','').replace('[','')           
        count +=1

In [ ]:
polygons = [Polygon(coords_total_mun[i]) for i in range(coordenadas_cidades[coordenadas_cidades['CD_GEOCMU'] == '3304557'].index[0], coordenadas_cidades[coordenadas_cidades['CD_GEOCMU'] == '3304557'].index[-1]+1)]
cidade_polygons = cascaded_union(polygons)
cidade_polygons

In [ ]:
all_cities_coords = list()
for i in range(0, coordenadas_cidades.shape[0]):
    coords_city = coordenadas_cidades['COORDENADAS'][i].split(',')
    city = list()
    for item in coords_city:
        city.append(float(item))
        
    all_cities_coords.append(city)
    print(str(100*i/coordenadas_cidades.shape[0]),end='\r')

In [ ]:
coords_total_mun =  list()
cities_coords = list()
k = 0 
while k <  len(all_cities_coords):
    p = 0
    cities_coords = list()
    while p < len(all_cities_coords[k]):
        x = (all_cities_coords[k][p],all_cities_coords[k][p+1] )
        p = p+2
        cities_coords.append(x)
    coords_total_mun.append(cities_coords)
    k = k+1

In [ ]:
resultados = pd.DataFrame('CNPJBAS','COD_MUN','ESTADO','CIDADE','NX','NY')
start_time = time.time()
base_testes = pd.DataFrame['CNPJBAS','CD_GEOCMU','NX','NY']

count = 0
for k in range(0, len(base_testes)):    
    polygons = [Polygon(coords_total_mun[i]) for i in range(coordenadas_cidades[coordenadas_cidades['CD_GEOCMU'] == str(base_testes.CD_GEOCMU[k])].index[0],
                                                            coordenadas_cidades[coordenadas_cidades['CD_GEOCMU'] == str(base_testes.CD_GEOCMU[k])].index[-1]+1)]
    cidade_polygons = cascaded_union(polygons)

    local = Point(float(dados_mun.LONG[k]), float(dados_mun.LAT[k]))

    if cidade_polygons.intersects(local) or cidade_polygons.contains(local):
        n_ = 'n'
    else:
        resultados.loc['CNPJBAS'] = base_testes.CNPJBAS[k]
        resultados.loc['COD_MUN'] = base_testes.COD_MUN[k]
        resultados.loc['ESTADO'] = base_testes.ESTADO[k]
        resultados.loc['CIDADE'] = base_testes.CIDADE[k]
        resultados.loc['NX'] = base_testes.NX[k]
        resultados.loc['NY'] = base_testes.NY[k]
        count +=1
        
print("--- %s seconds ---" % (time.time() - start_time))  

In [ ]:
#teste se está dentro do município informado
# resultados = pd.DataFrame()
# start_time = time.time()
# for k in range(0, len(dados_mun)):
#     index_coordenada_mun = data_final.loc[ data_final['CD_GEOCMU'] == str(dados_mun.CD_GCMUN[k]) ].index[0]

#     polygon = Polygon(coords_total_mun[ index_coordenada_mun ])
#     local = Point(float(dados_mun.LONG[k]), float(dados_mun.LAT[k]))

#     if polygon.intersects(local) or polygon.contains(local):
#         resultados = resultados.append([[dados_mun.CD_GCMUN[k],data_final.CD_GEOCMU[index_coordenada_mun]]])
#     else:
#         continue
# print("--- %s seconds ---" % (time.time() - start_time))   

In [ ]:
resultados.to_excel('resultados.xlsx', encoding='latin-1')

In [ ]:
#merge
k = 0
final_csv = pd.DataFrame()
while k <=17:
    final_csv = final_csv.append(pd.read_csv('chunk'+str(k)+'.csv',index_col=0))
    k=k+1
final_csv = final_csv.drop(final_csv.columns[0], axis=1)

In [ ]:
final_csv.head()